# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/credit_scoring_eng.csv') #open and read the dataframe file 
df.info() #dispaly the data general information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [3]:
for i in df: #creating function to check on the dataset columns if they contain 0 values
    print(i, len(df[df[i]==0])) 

children 14149
days_employed 0
dob_years 101
education 0
education_id 5260
family_status 0
family_status_id 12380
gender 0
income_type 0
debt 19784
total_income 0
purpose 0


In [4]:
df['education'].value_counts() 

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [5]:
df['children'].unique() #display the unique values of 'children' column

array([ 1,  0,  3,  2, -1,  4, 20,  5])

### Conclusion

After reading the file and displaying the general information, I noticed there are two columns with missing values, 'days_employed' and 'total_income' and they are the only columns with the 'float64' value type.

Therefore, I looked deeper into the data for those columns the majority of the missed data is due to unemployed or retired people, they have no total for their income and no employment days. As a first look at the data frame, those were the first two things that I noticed. Nevertheless, the 'days_employed' column has a lot of messy data. 

I started to look for 0 values in the data frame, and if they are relevant to be in certain columns or not, for example, it was relevant to find 0 values in the 'children' column but it wasn't relevant to find them in the 'dob_years' column. 

I also noticed the duplicates in the 'education' column with different text formats. Therefore, I printed the values of the 'education' column.

I checked the unique values of the 'children' column, and apparently, there is a typo in one of the values that is '-1'. 

In the next step, I will clean up the data and fix the missing and irrelevant values.

## Data preprocessing

### Processing missing values

In [6]:
df = df.drop(columns='days_employed')  # deleting the "days_employed" column 

In [7]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median')) #filling the missing values with the median of each 'income_type' in the 'total_income' column 

In [8]:
df.loc[df['children'] < 0, 'children'] = 1 # to replace the negative amout of children '-1' to be '1'  i believe it was a typo
df = df.drop(df[df['children'] > 5].index) #keep the range of children between 0 to 5, and drop the rest 

In [9]:
avg_age = int(df['dob_years'].mean()) # measure the median of the ages in the dataset 
df['dob_years'] = df['dob_years'].replace(0, avg_age) # replace the 0 values with the median of the age dataset 

In [10]:
display(df.head(10)) #display the first 10 rows in the dataframe 

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [11]:
df.isnull().sum() #check if we have any missing values left

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Conclusion

I decided to drop the column of 'days_employed' since it does not need the project tasks, and also, the data in it is super messy. Although it could be cleaned and, arranged it's better to focus on the project tasks.

Since I deleted the 'days_employed' column, the only column left with missing values is the 'total_income'. 

In the 'total_income' I filled the missing value with the median of income for each 'income_type', so if the 'income_type' of the missing value is Business, it will calculate the median for business 'income_type' and place it in the 'total_income'.

The 'children' column had a -1 value, where it wasn't relevant. I believe it was a typo during the data entry. Therefore, I used the loc method with a condition statement to replace the value in a fast way. I also eliminated the value 20, I believe it wasn't relevant therefore I used the drop method to drop any value above 5 to keep the range of children between 0 and 5, and the only value that was above 5 is 20. 

There were irrelevant values in some columns like 'dob_years', it contained 0 values. 0 is an irrelevant value for age, so I measured the mean of all the 'dob_years' column and saved it in the avg_age variable and I used it to replace the 0's in the 'dob_years' column.

I display the data frame to take another look and I checked if we still have any missing value left, and according to the table, we don't have anymore.

### Data type replacement

In [12]:
df['total_income'] = df['total_income'].astype(int) #convert the total_income type to integer 
display(df.head())

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,33,Secondary Education,1,married,0,M,employee,0,23341,purchase of the house
3,3,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


### Conclusion

Going back to 'total_income', it is the only column in the dataset that has the type of 'float64'. We need to need to convert the type to 'int'. I used the astype() method to convert all the values of the column to integer, and we can see in the table that the values look more organized and clean in the data frame.  

### Processing duplicates

In [13]:
df['education'] = df['education'].str.lower() # take all text to lowercase and delete duplicates 
display(df.head())
print(df['education'].value_counts())

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house
3,3,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


secondary education    15173
bachelor's degree       5246
some college             742
primary education        282
graduate degree            6
Name: education, dtype: int64


### Conclusion

For deleting duplicates, I used the str.lower() function on the 'education' column, which will take all the text format to lowercase, since the duplicates had the same names but in different formats of upper and lower cases. The str.lower() takes the string to lowercase and would join the similar strings together as we can see in the table. The categories are mentioned once and were merged. The duplicated could come up due to a text field that different employees fill manually. some could have the capslock activated, others could only type with lowercase some type with the first letter as caps. Therefore, that could be a convenient reason to have duplicated data.

### Categorizing Data

In [14]:
def loan_cat(value): #define a function to categorize the data (text) that will come as a value to it
    
    if 'hous' in value or 'real' in value or 'property'in value:
        return 'Real Estate'
    elif 'educ' in value or 'university' in value:
        return 'Education'
    elif 'wedding' in value:
        return 'Wedding'
    elif 'car' in value:
        return 'Car'
    return 'undefined'

df['purpose_cat'] = df['purpose'].apply(lambda value: loan_cat(value)) # taking the sentences from the df['purpose'] column and apply them to the loan_cat function and save the results in the df[purpose_cat] column
print(df['purpose_cat'].value_counts()) #printing the values of the column and we can see that all the rows are categorized. 

Real Estate    10804
Car             4299
Education       4007
Wedding         2339
Name: purpose_cat, dtype: int64


In [15]:
print(df['total_income'].describe())

count     21449.000000
mean      26434.989603
std       15698.088216
min        3306.000000
25%       17243.000000
50%       22815.000000
75%       31285.000000
max      362496.000000
Name: total_income, dtype: float64


In [16]:
def income_cat(amount): #define function to categorise the level of living for people depending on their income
    if amount < 15698:
        return 'Poor'
    elif amount < 26434:
        return 'Medium'
    else:
        return 'Rich'
df['income_cat'] = df['total_income'].apply(lambda amount: income_cat(amount))
print(df['income_cat'].value_counts())

Medium    9059
Rich      8161
Poor      4229
Name: income_cat, dtype: int64


In [17]:
df.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_cat,income_cat
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,Real Estate,Rich
1,1,36,secondary education,1,married,0,F,employee,0,17932,car purchase,Car,Medium
2,0,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,Real Estate,Medium
3,3,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,Education,Rich
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,Wedding,Medium
5,0,27,bachelor's degree,0,civil partnership,1,M,business,0,40922,purchase of the house,Real Estate,Rich
6,0,43,bachelor's degree,0,married,0,F,business,0,38484,housing transactions,Real Estate,Rich
7,0,50,secondary education,1,married,0,M,employee,0,21731,education,Education,Medium
8,2,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337,having a wedding,Wedding,Poor
9,0,41,secondary education,1,married,0,M,employee,0,23108,purchase of the house for my family,Real Estate,Medium


### Conclusion

Checking through the 'purpose' column there are a lot of similar purposes for loans but, with a different type of text. most of the time it is written in a sentence and not in one word. we cannot categorize this data as we did in the 'education' column simply because the sentences are not similar from a text perspective. Therefore, we need to find a common word between sentences so, it will allow us to categorize the data in the column.

I created a function, 'loan_cat' that will check if the sentences in the column contain certain words. The function contains an if statement that will return at the end a result of 4 categories: Real Estate, Car, Education, and Wedding. 

The results would be saved in the 'purpose_cat' column after applying the 'total_income' column to the function 'loan_cat'.

The next column I categorized as the 'total_income'. I printed the description for the values in the 'total_income', and I compared the results with small google research about the minimum wage in general. The amount that I found was close to the std amount in the description table, and the medium wage was also similar to the mean in the table. 
Therefore, I created an if statement to check the amount of the 'total_income' in each row and gives the category output in the new column 'income_cat'.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [18]:
pivot_1= pd.pivot_table(df, index ='children', columns='debt', values='family_status_id', aggfunc='count', margins= True).reset_index()
pivot_1['ratio']= pivot_1[1] / pivot_1['All']*100
pivot_1

debt,children,0,1,All,ratio
0,0,13086.0,1063.0,14149,7.512898
1,1,4420.0,445.0,4865,9.146968
2,2,1861.0,194.0,2055,9.440389
3,3,303.0,27.0,330,8.181818
4,4,37.0,4.0,41,9.756098
5,5,9.0,NaN,9,NaN
6,All,19716.0,1733.0,21449,8.079631


### Conclusion

The relation between paying the loan on time and having kids as shown in the table above, the lowest percentage of people that are not repaying the loan on time, they are the families who have no children with the percentage of 7.51%. Where is the average, in general, is 8.07%. And the highest percentage of families who don't repay the loan on time are the families who have 4 children with a percentage of 9.75%. 

Therefore, it is clear that families have fewer children of none have a higher chance to repay the loan on time. 

- Is there a relation between marital status and repaying a loan on time?

In [19]:
pivot_2= pd.pivot_table(df, index ='family_status', columns='debt', values='dob_years', aggfunc='count', margins= True).reset_index()
pivot_2['ratio']= (pivot_2[1] / pivot_2['All'])*100
display(pivot_2)

debt,family_status,0,1,All,ratio
0,civil partnership,3780,385,4165,9.243697
1,divorced,1109,84,1193,7.041073
2,married,11403,928,12331,7.525748
3,unmarried,2531,273,2804,9.736091
4,widow / widower,893,63,956,6.589958
5,All,19716,1733,21449,8.079631


### Conclusion

The total average of paying the loan on time and family status is 8.07%. The highest amount of people who wouldn't repay the loan on time are the people who are unmarried with the highest percentage of 9.73%. While the lowest percentage of 6.56% for people who repay their loan is a widow or a widower.

we can see from this that people who have fewer responsibilities are more irresponsible about repaying the loan meanwhile, people with life responsibilities have a higher percentage to repay the loan on time. 

- Is there a relation between income level and repaying a loan on time?

In [20]:
pivot_3= pd.pivot_table(df, index ='income_cat', columns='debt', values='income_type', aggfunc='count', margins= True).reset_index()
pivot_3['ratio']= pivot_3[1] / pivot_3['All']*100
display(pivot_3)

debt,income_cat,0,1,All,ratio
0,Medium,8288,771,9059,8.510873
1,Poor,3887,342,4229,8.087018
2,Rich,7541,620,8161,7.597108
3,All,19716,1733,21449,8.079631


### Conclusion

Categorizing the income made it clearer for us to understand the people level who would repay the loan on time. regarding the table, an interesting result shows that the percentage of the medium class would have a higher rate than the poor class in not repaying the loan in time by almost 0.5%. 

If we look at the bigger image we won't see a huge difference between the percentages of the three categories the biggest difference between them is still less than 1%. Therefore, The people's life level is less making an effect on the loan repayment.

- How do different loan purposes affect on-time repayment of the loan?

In [21]:
pivot_4= pd.pivot_table(df, index ='purpose_cat', columns='debt', values='total_income', aggfunc='count', margins= True).reset_index()
pivot_4['ratio']= pivot_4[1] / pivot_4['All']*100
display(pivot_4)

debt,purpose_cat,0,1,All,ratio
0,Car,3898,401,4299,9.327751
1,Education,3638,369,4007,9.208884
2,Real Estate,10024,780,10804,7.219548
3,Wedding,2156,183,2339,7.823856
4,All,19716,1733,21449,8.079631


### Conclusion

The shows how the purpose of the loan would affect the time of repaying the loan. The total average of all purposes is 8.07%. While the highest percentage is 9.32% for loans that go to cars, those are harder to repay the loan on time. The lowest percentage goes to Real Estate. 
Weddings and Real Estate are getting repaid better than Cars and Education. Again, I think that is all related to Age and the responsibility that people have. 

## General conclusion

In this project, I checked the file of the data frame to understand the type of data that I need to work with and what kind of information I can pull out of it.  I started to read the general information about the data frame, where I noticed the missing values for the 'days_employed' and 'total_income' columns. 

After taking notes about those two columns, I checked the data frame for any 0 values and if they are relevant to where they exist. Meaning it was relevant to find them in the 'children' column, but irrelevant to find them in the 'dob_year' column. I made a research on all the columns and I found out the following: 
The data of the column 'days_employed' is messy and irrelevant to the project.
the 'total_income' has missing values.
'dob_years' column has 0 values and it needs to be replaced.
The 'education' column, has a lot of duplicated and unorganized text formats.
There is a negative value in the 'children' column, and that's irrelevant. 
First, I dropped the 'days_employed' column from the data frame, since it has no use in this project. The data could be fixed although, it would be still not relevant to the project tasks. This type of data should be reported to the engineers to understand that there might be something wrong since it has a lot of messy information and is not clear. 
Next, I filled the missing values in the 'total_income' column by the median of the 'income_type'. That means if the work is civil servant, the missing value for a civil servant would be the median of all the civil servant incomes of the non-missing values. That also applied to all the 'income_type' categories. 

I calculated the mean of the 'dob_years' and I replaced the 0 values with it. 
The negative value in the 'children' column I replaced with the 1 value because apparently, the value was a typo during the data entry.
 Now I finished the data preprocessing, but I checked once again if there are any missing values, and regarding the results, there were no missing values. 
The 'total_income' is a 'float64' data type, I converted it to an integer by using the astype() method. Converting the column to an integer would make it easier to read and to work within the matter of calculations. 
As I said before, the 'education ' had a lot of duplicated data and messy text formats. Therefore, I applied the str. lower() method that would take all the strings to a lowercase text and will combine the duplicates. So, I ended up having 5 categories only in the 'education' column.
After cleaning the data and filling in the missing values, I needed to categorize the following columns:
'purpose' column: The loan purpose column contains sentences that tell what is the reason for getting a loan. The problem here is the similarity in the purpose but not in the text. However, there are common words that I could notice between the purposes. 
'income_total' column: The income column has a huge range between the numbers of income. That makes it hard to have a clearer image of the effects of income on the loan. Therefore, the best way is to have small and specific categories for certain ranges in income. 
In the 'purpose' column I applied a function to read the text from it and compare the text if it has certain words in it. I used the If statement, I saw it easier and faster to build, since we don't have a huge amount of text. Then the results were saved on the 'purpose_cat' column. The result was 4 categories: Real Estate, Car, Education and Wedding. 

I made a small research on google to know more about the general amount of minimum wage. That to help me categorize the 'total_income' column. However, I also printed the describe() table of the column. Therefore, I can get to know more about the content. The minimum wage was close enough to the std in the table, and the medium wage was relevant to the mean amount in the table. Therefore, I made an if statement to check the 'total income' and categorize it by either: Poor, Medium, or Rich, and it was saved on the 'income_cat' column. 

Now we have a clean data set and categorized, for the last section I needed to answer some questions to analyze the data. 
In all those questions, I used the Pivot table, to check relations between values in different columns. Then I calculated the ratio to get the actual percentage.
 I will answer the questions briefly here since I answered them in detail in the previous section. 

The Questions:  
Is there a relation between having kids and repaying a loan on time?

Families with 4 children have a percentage of 9.75% to not pay on time. 
Families with no children have a percentage of 7.51% to not pay on time.

Is there a relation between marital status and repaying a loan on time?

Unmarried with the highest percentage of 9.73% to not pay on time.
A widow or a widower with the lowest percentage of 6.56% to not pay on time.

Is there a relation between income level and repaying a loan on time?

The medium level has the highest percentage to not pay on time 8.51%
The rich level has the lowest percentage to not pay on time 7.59%

How do different loan purposes affect on-time repayment of the loan?

The Car purpose has 9.32% chance to not pay the loan on time
The real estate purpose has 7.21% chance to not pay the loan on time. 
